In [10]:
import numpy as np
import pickle
import cv2
import torch
import torchvision.transforms as transforms
from torchvision.models import resnet50
from sklearn.feature_extraction.text import TfidfVectorizer

In [11]:
# Load and preprocess images
def preprocess_images(image_paths):
    preprocessed_images = []
    for path in image_paths:
        image = cv2.imread(path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)  # Convert BGR to RGB
        image = cv2.resize(image, (224, 224))  # Resize to the required input size for ResNet
        preprocessed_images.append(image)
    return np.array(preprocessed_images)

# Load ResNet50 model pretrained on ImageNet
resnet_model = resnet50(pretrained=True)
resnet_model = resnet_model.eval()

# Extract features using ResNet50
def extract_resnet_features(images):
    transform = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])
    preprocessed_images = torch.stack([transform(image) for image in images])
    with torch.no_grad():
        features = resnet_model(preprocessed_images)
    return features.numpy()

# Normalize extracted features
def normalize_features(features):
    norms = np.linalg.norm(features, axis=1, keepdims=True)
    normalized_features = features / norms
    return normalized_features

c:\Users\HP\Documents\Shrugal IIITD\Semester 8\Information Retrieval\CSE508_Winter2024_A2_2020408\myenv\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\HP\Documents\Shrugal IIITD\Semester 8\Information Retrieval\CSE508_Winter2024_A2_2020408\myenv\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to C:\Users\HP/.cache\torch\hub\checkpoints\resnet50-0676ba61.pth
100.0%


In [12]:
# Sample paths to images
image_paths = ["image1.jpg", "image2.jpg", "image3.jpg"]  # Replace with actual image paths

In [13]:
# Preprocess images
preprocessed_images = preprocess_images(image_paths)

error: OpenCV(4.9.0) D:\a\opencv-python\opencv-python\opencv\modules\imgproc\src\color.cpp:196: error: (-215:Assertion failed) !_src.empty() in function 'cv::cvtColor'


In [ ]:
# Extract features using ResNet50
resnet_features = extract_resnet_features(preprocessed_images)

In [ ]:
# Normalize extracted features
normalized_features = normalize_features(resnet_features)

In [ ]:
# Save extracted features
with open('extracted_features.pkl', 'wb') as f:
    pickle.dump(normalized_features, f)

In [ ]:
# TF-IDF scores (just for demonstration, replace with your own implementation)
tfidf_scores = {'image1.jpg': [0.2, 0.5, 0.3], 'image2.jpg': [0.1, 0.2, 0.7], 'image3.jpg': [0.6, 0.1, 0.3]}

In [ ]:
# Save TF-IDF scores
with open('tfidf_scores.pkl', 'wb') as f:
    pickle.dump(tfidf_scores, f)